# Basic simulation of electrodes in ESPResSo part I:
# Ion-pair in a narrow metallic slit-like confinement using ICC $\star$

## Prerequisites

To work with this tutorial, you should be familiar with the following topics:

-  Setting up and running simulations in ESPResSo - creating particles,
   incorporating interactions.
   If you are unfamiliar with this, you can go through the respective tutorial
   in the `lennard_jones` folder.
-  Basic knowledge of classical electrostatics:
   Dipoles, surface and image charges
-  Reduced units, as described in the **ESPResSo** [user guide](https://espressomd.github.io/doc/introduction.html#on-units)

## Introduction

This tutorial introduces some basic concepts for simulating charges close to an
electrode interface using **ESPResSo**.
In this first part, we focus on the interaction of a single ion pair confined in
a narrow metallic slit pore using the ICC $\star$-algorithm
<a href='#[1]'>[1]</a> for the computation of the surface polarization.
Here, we verify the strong deviation from a Coulomb-like interaction:
In metallic confinement, the ion pair interaction energy is screened
exponentially due to the presence of induced charges on the slit walls.
<a href='#[1]'>[2]</a>  

## Theoretical Background 

The normal component of electric field across a surface dividing two dielectric
media yields a discontinuity, which can be expressed in terms of a finite
surface charge density 
$(\epsilon_1\vec{E}_1 - \epsilon_2\vec{E}_2).\hat{n}=-\sigma(\vec{r})$.
This expression describes the jump in the electric field across the material
interface going from a dielectric medium $\epsilon_1$ to another one,
$\epsilon_2$.

While in the case of non-polarizable materials ($\epsilon_1 = \epsilon_2 = 1$),
this jump is only related to surface charges and dielectric contrast and the
potential is continuous across the interface, for polarizable materials also the
polarization field $\vec{P}$ will give a contribution. 
In order to solve the problem in presence of a jump of the dielectric constant
across an interface, one must know the electric fields on both sides. 

Another approach is to replace this two domain problem by an equivalent one
without the explicit presence of the dielectric jump.
This is achieved by introducing an additional fictitious charge i.e. an induced
charge density $\sigma_{ind}$ on the surface. 
With this well known "method of image charges", it is sufficient to know the
electric field on one side of the interface. 
**ESPResSo** provides the "Induced Charge Calculation with fast Coulomb Solvers"
*(ICC  $\star$) algorithm <a href='#[1]'>[1]</a> which employs a  numerical
*scheme for solving the boundary integrals and the induced charge.   

*Note*: Apart from ICC $\star$, **ESPResSo** offers the "Electrostatic layer
correction with image charges" (ELC-IC) method <a href='#[3]'>[3]</a>, for
planar 2D+h partially periodic systems with dielectric interfaces.
The tutorial on *Basic simulation of electrodes in ESPResSo part II*
addresses this in detail.

The Green's function for two point charges in a dielectric slab can be solved
analytically <a href='#[2]'>[2]</a>.
In the metallic limit the dielectric contrast is
$$ \Delta = \frac{\epsilon_1 - \epsilon_2} {\epsilon_1 + \epsilon_2} = -1 .$$
If the ions are placed in the center of the slab of width $w$ and a distance $r$
away from each other, the Green's function accounting for all image charges
simplifies to <a href='#[4]'>[4]</a> 
$$ 4 \pi \epsilon_0 \epsilon_r w \mathcal{G}(x) = \sum_{n=-\infty}^\infty \frac{-1^n}{\sqrt{x^2+n^2}} ,$$
where we have introduced the scaled separation $x = r/w$.

For $x\to 0$ the term for $n = 0$ dominates and one recovers
$$ \lim_{x\to 0} 4 \pi \epsilon_0 \epsilon_r w \mathcal{G}(x) = \frac{1}{x},$$
which is the classical Coulomb law.
Contrary, for large distances $x \to \infty$ one finds
$$ \lim_{x\to \infty} 4 \pi \epsilon_0 \epsilon_r w \mathcal{G}(x) = \sqrt{\frac{8}{x}} e^{-\pi x},$$
i.e. the interaction decays exponentially.
Such screened electrostatic interactions might explain unusual features
concerning the nano-confined ionic liquids employed for supercapacitors referred
to 'super-ionic states'.

We will explore this interaction numerically using ICC $^\star$ in the following.

## 2D+h periodic systems, dielectric interfaces and Induced Charge Computation with ICC $\star$

Partially periodic ionic systems with dielectric interfaces are very often
encountered in the study of energy materials or bio-macromolecular and membrane
studies. 
These systems usually exhibit a confinement along one ($z$) direction, where the
confining boundary or interface imposes a dielectric discontinuity, while the
other $x$-$y$ directions are treated periodic. 
To such a partially periodic system, we combine the efficient scaling behavior
of three-dimensional mesh-based solvers (typically
$\mathcal{O}(N \log N)$ for P3M) with the Electrostatic Layer Correction (ELC)
<a href='#[3]'>[3]</a>.
The latter corrects for the contributions from the periodic images in the
constrained direction and its numerical cost grows linear with the number of
point charges $N$, hence the performance overall depends on the underlying $3-D$
Coulomb solver.
The method relies on an empty vacuum slab in the simulation box in the
$z$-direction perpendicular to slab.
While in theory this can become relatively small (typically 10% of the box
length), its choice in practice strongly affects the performance due to the
tuning of the P3M parameters to obtain the desired accuracy.

We furthermore employ ICC $\star$ to solve the Poisson equation for an
inhomogeneous dieletric:
$$ \nabla (\epsilon \nabla \phi)=-4\pi \rho$$

The image charge formalism can be derived as follows:
- Integrate the latter expression at the boundary over an infinitesimally wide
pillbox, which will give the induced surface charge in this infinitesimal
segment as (Gauss law):
$$q_{ind} = \frac{1}{4\pi} \oint\, dA\, \cdot \epsilon\nabla \phi = \frac{A}{4\pi}(\epsilon_1\vec{E}_1 \cdot \hat{n}-\epsilon_2\vec{E}_2 \cdot\hat{n})$$
- The electric field at the closest proximity of the interface, $\vec{E}_{1/2}$,
can be written as a sum of electric field contributions from the surface charge
$\sigma$ and the external electric field $\vec{E}$:
$$ \vec{E}_{1/2} =\vec{E} + 2\pi/\epsilon_1\sigma\hat{n}  $$
- Combining this with the previous expression, the induced charge can be written in terms of the dielectric mismatch $\Delta$ and the electric field as:
$$\sigma = \frac{\epsilon_1}{2\pi} \frac{\epsilon_1-\epsilon_2}{\epsilon_1+\epsilon_2}\vec{E} \cdot \hat{n} =: \frac{\epsilon_1}{2\pi} \Delta \, \vec{E} \cdot \hat{n}$$


The basic idea of the ICC $^\star$ formalism now is to employ a discretization
of the surface by means of spatially fixed ICC particles.
The charge of each ICC particle is not fixed but rather iterated using the
expressions for $\vec{E}_{1/2}$ and $\sigma$ above until a self-consistent
solution is found.

##  1. System setup 


We first import all ESPResSo features and external modules.

In [ ]:
import espressomd
import numpy as np
import espressomd.electrostatics
import espressomd.electrostatic_extensions
from espressomd.interactions import *

espressomd.assert_features(['ELECTROSTATICS'])

import matplotlib.pyplot as plt
from scipy.special import *
from tqdm import tqdm

We need to define the system dimensions and some physical parameters related to
length, time and energy scales of our system.
All physical parameters are defined in reduced units of length ($\sigma=1$;
Particle size), mass ($m=1$; Particle mass), time ($t=0.01 \tau$) and
elementary charge ($e=1$).

Another important length scale is the Bjerrum Length, which is the length at
which the electrostatic energy between two elementary charges is comparable to
the thermal energy $k_\mathrm{B}T$ and thus defines the energy scale in our
system.
It is defined as
$$\ell_\mathrm{B}=\frac{1}{4\pi\epsilon_0\epsilon_r k_\mathrm{B}T}.$$ 
In our case, if we choose the ion size ($\sigma$) in simulations to represent a
typical value for mono-atomar salt, 0.3 nm in real units, then the
Bjerrum length of water at room temperature, $\ell_\mathrm{B}=0.71 \,\mathrm{nm}$ is
$\ell_\mathrm{B}\sim 2$ in simulations units.

In [ ]:
#***************************************************
#            System Setup
#***************************************************

# Box dimensions
# To construct a narrow slit Lz << (Lx , Ly)
box_l_x = 100.
box_l_y = 100.
box_l_z = 5.

# ICC* with ELC: 2D electrostatics
ELC_GAP = 6*box_l_z

system = espressomd.System(box_l=[box_l_x, box_l_y, box_l_z+ELC_GAP])

# System Time
system.time_step = 0.01
system.cell_system.skin = 0.4

# Elementary charge 
q = np.array([1.0])  

# Interaction Parameters for P3M with ELC

BJERRUM_LENGTH = 2.0        # Electrostatic prefactor passed to P3M ; prefactor=lB KBT/e2                
ACCURACY = 1e-7             # P3M force accuracy      
CHECK_ACCURACY = 1e-7       # maximim pairwise error in ELC

#Lennard-Jones  Parameters

LJ_SIGMA = 1.0
LJ_EPSILON = 1.0                

#Particle parameters

types = {"Cation": 0, "Anion": 1  ,"Electrodes": 2}
charges = {"Cation": q[0], "Anion": -q[0]  }

p1=system.part.add(pos=[box_l_x/4.0, box_l_y/2.0, box_l_z/2.0], q=charges["Cation"])
print(f"Cation placed at position: {p1.pos}")
p2=system.part.add(pos=[3.0*box_l_x/4.0, box_l_y/2.0, box_l_z/2.0], q=charges["Anion"])
print(f"Anion placed at position: {p2.pos}")


### Task:
Set up electrostatics interactions between the ion pairs using P3M and ICC $\star$.  
#### Hints:
- First instantiate a P3M object (using
  [*espressomd.electrostatics.P3M(...)*](https://espressomd.github.io/doc/espressomd.html#espressomd.electrostatics.P3M))
  as the 3D Coulomb solver for the system.
- Since, we are working on a 2D+h partially periodic system, we use ELC in
  combination with P3M. For this purpose, instantiate ELC (using
  [*espressomd.electrostatics.ELC(...)*](https://espressomd.github.io/doc/espressomd.html#espressomd.electrostatics.ELC))
  with p3m as actor, which calls necessary initialization routines.  
- Now we account for dielectric interface by adding ICC $^\star$.
  To this end, first create an ICC object using
  [*espressomd.electrostatic_extensions.ICC(...)*](https://espressomd.github.io/doc/espressomd.html#espressomd.electrostatic_extensions.ICC).
- Setting up ICC $^\star$ requires specifying a fixed number of ICC particles
  and their initial charges.
  Since the ICC particles are normal ESPResSo particles, they need to be added
  at the interfaces using
  [*system.part.add(..)*](https://espressomd.github.io/doc/espressomd.html?#module-espressomd.particle_data).

*Note* - Refer to section
[**Dielectric interfaces with the ICC algorithm**](https://espressomd.github.io/doc/electrostatics.html#dielectric-interfaces-with-the-icc-star-algorithm)
in the **ESPResSo** documentation for the basics of an ICC $^\star$ setup.


In [ ]:
p3m = espressomd.electrostatics.P3M(
            prefactor=BJERRUM_LENGTH,
            accuracy=ACCURACY,
            check_neutrality = False,
            mesh = [100,100,150],
            cao = 5
        )

elc = espressomd.electrostatics.ELC(actor=p3m, gap_size=ELC_GAP, maxPWerror=CHECK_ACCURACY)

# Set the ICC line density and calculate the number of
# ICC particles according to the box size
nicc = 100  # linear density
nicc_per_electrode = nicc**2  # surface density
nicc_tot = 2 * nicc_per_electrode
iccArea = box_l_x * box_l_y / nicc_per_electrode
lx = box_l_x / nicc
ly = box_l_y / nicc

# Lists to collect required parameters
iccNormals = []
iccAreas = []
iccSigmas = []
iccEpsilons = []

# Add the fixed ICC particles:

# Bottom electrode (normal [0, 0, 1])
for xi in range(nicc):
    for yi in range(nicc):
        system.part.add(pos=[lx * xi, ly * yi, 0.], q=-0.0001,
                        type=types["Electrodes"], fix=[True, True, True])
iccNormals.extend([0, 0, 1] * nicc_per_electrode)

# Top electrode (normal [0, 0, -1])
for xi in range(nicc):
    for yi in range(nicc):
        system.part.add(pos=[lx * xi, ly * yi, box_l_z], q=0.0001,
                        type=types["Electrodes"], fix=[True, True, True])
iccNormals.extend([0, 0, -1] * nicc_per_electrode)

# Common area, sigma and metallic epsilon
iccAreas.extend([iccArea] * nicc_tot)
iccSigmas.extend([0] * nicc_tot)
iccEpsilons.extend([100000] * nicc_tot)

iccNormals = np.array(iccNormals, dtype=float).reshape(nicc_tot, 3)

icc = espressomd.electrostatic_extensions.ICC(
    first_id=2,
    n_icc=nicc_tot,
    convergence=1e-3,
    relaxation=0.95,
    ext_field=[0, 0, 0],
    max_iterations=1000,
    eps_out=1.0,
    normals=iccNormals,
    areas=np.array(iccAreas, dtype=float),
    sigmas=np.array(iccSigmas, dtype=float),
    epsilons=np.array(iccEpsilons, dtype=float)
    )

system.electrostatics.solver = elc
system.electrostatics.extension = icc

## 2. Calculation of the forces

In [ ]:
r = np.logspace(0,box_l_z/4.,10) 
elc_forces_axial = np.empty((len(r), 2))

for i, x in enumerate(tqdm(r)):
    p1.pos = [0, box_l_y/2.0, box_l_z/2.0]
    p2.pos = [x, box_l_y/2.0, box_l_z/2.0]

    system.integrator.run(0)
    elc_forces_axial[i, 0] = p1.f[0]
    elc_forces_axial[i, 1] = p2.f[0]
    
    # reset ICC charges to ensure charge neutrality check passes
    system.part.by_ids(range(2,2+nicc_per_electrode)).q = np.array([-0.0001]*nicc_per_electrode)
    system.part.by_ids(range(2+nicc_per_electrode,2+2*nicc_per_electrode)).q = np.array([0.0001]*nicc_per_electrode)    

## 3. Analysis and Interpretation of the data

With this we can now compare the force between the two ions to the analytical prediction.
To evaluate the infinite series we truncate at $n=1000$, which already is well converged.

In [ ]:
def analytic_force_centered(r,w):
    x = r/w
    prefactor = BJERRUM_LENGTH / w**2

    def summand(x, n):
        return (-1)**n * x/(x**2+n**2)**(3./2)
    
    def do_sum(x):
        max = int(1e3)
        sum = 0
        for n in range(-max+1,max+1):
            sum += summand(x,n)
        return sum

    F = do_sum(x) * prefactor
    return F

def coulomb_force(x):
    prefactor = BJERRUM_LENGTH
    E = prefactor / x**2
    return E

def exponential_force(r,w):
    x = r/w
    prefactor = BJERRUM_LENGTH
    E = prefactor * np.sqrt(2) * (1/x)**(3/2) * np.exp(-np.pi*x)
    return E

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.plot(r/BJERRUM_LENGTH, -elc_forces_axial[:,1],  color='red' , label="sim (p2)",  marker='o', ls='')
plt.plot(r/BJERRUM_LENGTH, elc_forces_axial[:,0],  color='k' , label="sim (p1)",  marker='x', ls='')

x = np.logspace(-.25,1.45,100)

plt.plot(x/BJERRUM_LENGTH, analytic_force_centered(x,box_l_z),  color='b' , label="analytic",  marker='')
plt.plot(x/BJERRUM_LENGTH, coulomb_force(x), color='green', ls='--', label='Coulomb')
plt.plot(x/BJERRUM_LENGTH, exponential_force(x,box_l_z), color='red', ls='--', label='Exponential')

plt.xlabel(r'$r \, [\ell_\mathrm{B}]$')
plt.ylabel(r'$F \, [k_\mathrm{B}T / \sigma$]')
plt.loglog()
plt.legend()

## References

<a id='[1]'></a>[1] Tyagi, S.; Süzen, M.; Sega, M.; Barbosa, M.; Kantorovich, S. S.; Holm, C. An Iterative, Fast, Linear-Scaling Method for Computing Induced Charges on Arbitrary Dielectric Boundaries. J. Chem. Phys. 2010, 132 (15), 154112. https://doi.org/10.1063/1.3376011.
 
<a id='[2]'></a>[2] Kondrat, S.; Feng, G.; Bresme, F.; Urbakh, M.; Kornyshev, A. A. Theory and Simulations of Ionic Liquids in Nanoconfinement. Chem. Rev. 2023, 123 (10), 6668–6715. https://doi.org/10.1021/acs.chemrev.2c00728.

<a id='[3]'></a>[3] Tyagi, S.; Arnold, A.; Holm, C. Electrostatic Layer Correction with Image Charges: A Linear Scaling Method to Treat Slab 2D+h Systems with Dielectric Interfaces. J. Chem. Phys. 2008, 129 (20), 204102. https://doi.org/10.1063/1.3021064.

<a id='[4]'></a>[4] Loche, P.; Ayaz, C.; Wolde-Kidan, A.; Schlaich, A.; Netz, R. R. Universal and Nonuniversal Aspects of Electrostatics in Aqueous Nanoconfinement. J. Phys. Chem. B 2020, 124 (21), 4365–4371. https://doi.org/10.1021/acs.jpcb.0c01967.